In [ ]:
!pip install 'aif360[LFR]' fairlearn gpflow

     |████████████████████████████████| 175 kB 14.5 MB/s 
     |████████████████████████████████| 177 kB 66.7 MB/s 
     |████████████████████████████████| 287 kB 63.9 MB/s 
     |████████████████████████████████| 462 kB 65.5 MB/s 
     |████████████████████████████████| 564 kB 66.2 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=659e5c21526cfd95f45f7b6774f022fd229fdef5aeb29af32e5998fabe9c2a34
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [ ]:
cd /usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult

/usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

--2022-02-20 20:37:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/x-httpd-php]
Saving to: ‘adult.data’

adult.data          100%[===================>]   3.79M  3.18MB/s    in 1.2s    

2022-02-20 20:37:34 (3.18 MB/s) - ‘adult.data’ saved [3974305/3974305]

--2022-02-20 20:37:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [application/x-httpd-php]
Saving to: ‘adult.names’

adult.names         100%[===================>]   5.11K  --.-KB/s    in 0s      

2022-

In [1]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras
import time
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels



import copy


from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.metrics import ClassificationMetric
from aif360.sklearn.metrics import equal_opportunity_difference
from aif360.sklearn.datasets import standardize_dataset

import gpflow

from gpflow.utilities import print_summary, set_trainable
from gpflow.ci_utils import ci_niter

import warnings
warnings.filterwarnings("ignore")

np.random.seed(0)
tf.random.set_seed(123)

In [54]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
adult_dataset = load_preproc_data_adult(['sex'])

#train, test = dataset_orig.split([0.7], shuffle=True, seed=3)
#print("training data size", train.features.shape)
#print("dataset feature names", train.feature_names)

#data_preprocessing = Pipeline([("standardiser", StandardScaler()),
                               #("minmaxscaler", MinMaxScaler()),
                                # ]
                                # )


adult_income_data, adult_dict = adult_dataset.convert_to_dataframe()

print(adult_income_data["sex"])

X, y = standardize_dataset(df=adult_income_data, prot_attr='sex', target='Income Binary')

print(type(X))
print(y)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=42
    )

0        1.0
1        1.0
2        1.0
3        1.0
4        0.0
        ... 
48837    0.0
48838    1.0
48839    0.0
48840    1.0
48841    0.0
Name: sex, Length: 48842, dtype: float64
<class 'pandas.core.frame.DataFrame'>
       sex
0      1.0    0.0
1      1.0    0.0
2      1.0    1.0
3      1.0    1.0
4      0.0    0.0
             ... 
48837  0.0    0.0
48838  1.0    1.0
48839  0.0    0.0
48840  1.0    0.0
48841  0.0    1.0
Name: Income Binary, Length: 48842, dtype: float64


In [3]:
# x_train = train.features
# x_test = test.features
# # x_train = data_preprocessing.fit_transform(train.features)
# # x_test = data_preprocessing.fit_transform(test.features)
# y_train = train.labels.ravel()
# y_test = test.labels.ravel()

#training_data = (x_train, y_train)


# N = len(x_train)
# M = 400
print("Number of training datapoints:", len(X_train))
print("Number of features", len(X.columns))

Number of training datapoints: 34189
Number of features 18


## Model Training

### Setting up the model

In [4]:
class SVGP_classifier(ClassifierMixin, BaseEstimator):
    """ A multiclass classifier that uses SVGP (Hensman, 2013).
    Parameters
    ----------
    demo_param : str, default='demo'
        A parameter used for demonstation of how to pass and store paramters.
    Attributes
    ----------
    X_ : ndarray, shape (n_samples, n_features)
        The input passed during :meth:`fit`.
    y_ : ndarray, shape (n_samples,)
        The labels passed during :meth:`fit`.
    classes_ : ndarray, shape (n_classes,)
        The classes seen at :meth:`fit`.
    """
    def __init__(self, M=100):
        self.M = M

        

        self.elbo_training_log = []
        

    def run_adam(self, model, iterations, training_data, minibatch_size):
        """
        Utility function running the Adam optimizer

        :param model: GPflow model
        :param interations: number of iterations
        """
        # Create an Adam Optimizer action
        train_dataset = tf.data.Dataset.from_tensor_slices(training_data).repeat().shuffle(self.N)
        logf = []
        train_iter = iter(train_dataset.batch(minibatch_size))
        training_loss = model.training_loss_closure(train_iter, compile=True)
        optimizer = tf.optimizers.Adam()

        @tf.function
        def optimization_step():
            optimizer.minimize(training_loss, model.trainable_variables)

        for step in range(iterations):
            optimization_step()
            if step % 10 == 0:
                elbo = -training_loss().numpy()
                logf.append(elbo/minibatch_size)
        return logf    


    def fit(self, X, y, minibatch_size=50, total_iterations=100):
        """A reference implementation of a fitting function for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples.
        y : array-like, shape (n_samples,)
            The target values. An array of int.
        Returns
        -------
        self : object
            Returns self.
        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        self.N = len(y)
        self.C = len(self.classes_)


        kernel_multi = gpflow.kernels.Matern52() + gpflow.kernels.White(variance=0.01)
        # Robustmax Multiclass Likelihood
        invlink = gpflow.likelihoods.RobustMax(self.C)  # Robustmax inverse link function
        likelihood_multi = gpflow.likelihoods.MultiClass(self.C, invlink=invlink)  # Multiclass likelihood
        Z = X[::self.M].copy()  # inducing inputs

        self.m = gpflow.models.SVGP(
            kernel=kernel_multi,
            likelihood=likelihood_multi,
            inducing_variable=Z,
            num_latent_gps=self.C,
            whiten=True,
            q_diag=True,
        )

        set_trainable(self.m.kernel.kernels[0].variance, False)
        set_trainable(self.m.inducing_variable, False)



        self.mb_size = minibatch_size
        training_data = (X, y)
        maxiter = ci_niter(total_iterations)
        self.elbo_training_log = self.run_adam(
            self.m,
            maxiter,
            training_data,
            self.mb_size)

        # Return the classifier
        return self


    def predict(self, X_test):
        """ A reference implementation of a prediction for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            The label for each sample is the label of the closest sample
            seen during fit.
        """
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        X_test = check_array(X_test)

        mu, _ = self.m.predict_y(X_test)
        predictions = []
        for probs in mu:
            prediction = np.argmax(probs)
            predictions.append(prediction)
        return predictions


In [ ]:

# class SVGP_classifier:
#     def __init__(self, num_of_classes=2, num_of_inducing_points=50,minibatch_size=50):
#         self.mb_size = minibatch_size
        
#         self.C = num_of_classes
        
#         self.mb_size = 50

#     def fit(self, X_train, y_train, minibatch_size=50, total_iterations=10000):      

#     def predict_targets(self, X_test):      

#     def get_params(self, deep=False):
#         return {'minibatch_size':self.mb_size}

### Training the models

In [93]:
from fairlearn.metrics import true_positive_rate

def tpr_differ(y_test, predictions):
    female_test = []
    male_test = []
    female_pred = []
    male_pred = []
    for i, index in enumerate(list(y_test.index)):
        if index[1]==0:
            female_test.append(y_test[index])
            female_pred.append(predictions[i])
        if index[1]==1:
            male_test.append(y_test[index])
            male_pred.append(predictions[i])
    tpr_male = true_positive_rate(male_test, male_pred)
    tpr_female = true_positive_rate(female_test, female_pred)
    return tpr_female-tpr_male


In [97]:
#mb_sizes = list(np.linspace(10, 20, 2, dtype=int))
##print(mb_sizes)

mb_sizes = [10, 20]

scoring_metrics = {"accuracy": "accuracy", "equal_opportunity_difference": make_scorer(tpr_differ)}

score_log = []

with tf.device('/device:GPU:0'):
    for mb_size in mb_sizes:
        svgpclass = SVGP_classifier(400)
        scores = cross_validate(svgpclass, X_train, y_train, scoring=scoring_metrics, cv=5, fit_params={"minibatch_size": mb_size, "total_iterations":100})
        print("Minibatch size:", mb_size, "\n", "average accuracy:",np.mean(scores["test_accuracy"]), "     Average equal opportunity difference:",np.mean(scores["test_equal_opportunity_difference"]))
        score_log.append(scores)


Minibatch size: 10 
 average accuracy: 0.7581970818161062      Average equal opportunity difference: 0.0
Minibatch size: 20 
 average accuracy: 0.7581970818161062      Average equal opportunity difference: 0.0


In [ ]:



# elbo_training_logs = []
# models = []


# with tf.device('/device:GPU:0'):
#     for mb_size in mb_sizes:
#         model = copy.deepcopy(m)
#         elbo_training_log = run_adam(model, maxiter, training_data, mb_size)
#         elbo_training_logs.append(np.array(elbo_training_log)/mb_size)
#         models.append(model)


# for log in elbo_training_logs:
#     plt.plot(range(0, total_iterations, 10), log)
#     plt.show()

In [ ]:





# def calculate_test_accuracy(model, x_test, y_test):
#     predictions = predict_targets(model, x_test)
#     total_correct = 0
#     for i in range(len(x_test)):
#         if predictions[i]==y_test[i]:
#             total_correct += 1

#     return total_correct/len(predictions)


# def logistic_regression_accuracy(training_data, x_test, y_test):
#     model = LogisticRegression()
#     model.fit(*training_data)
#     predictions = model.predict(x_test)
#     return accuracy_score(y_test, predictions)

# def mlp_accuracy(training_data, x_test, y_test):
#     model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#     model.fit(*training_data)
#     predictions = model.predict(x_test)
#     return accuracy_score(y_test, predictions)


# fairness_scores = []
# test_accuracies = []

# for i in range(len(models)):
#     test_accuracy = calculate_test_accuracy(models[i], x_test, y_test)
#     print(f"SVGP accuracy (minibatch = {mb_sizes[i]}):", test_accuracy)
#     test_accuracies.append(test_accuracy)
#     predictions = predict_targets(models[i], x_test)
#     test_pred = test.copy()
#     test_pred.labels = np.array(predictions)
#     fairness_metric = ClassificationMetric(
#         test, 
#         test_pred,
#         unprivileged_groups=unprivileged_groups, 
#         privileged_groups=privileged_groups
#         )
#     fairness_scores.append(fairness_metric.equal_opportunity_difference())
# print("Logistic Regression accuracy:", logistic_regression_accuracy(training_data, x_test, y_test))
# print("MLP accuracy:", mlp_accuracy(training_data, x_test, y_test))



In [ ]:
# plt.plot(mb_sizes, test_accuracies)
# plt.show()
# plt.plot(mb_sizes, -fairness_scores)
# plt.show()

In [5]:
clf_sv = SVGP_classifier()
clf_sv.fit(X_train, y_train)
predictions = clf_sv.predict(X_test)


2022-02-21 10:15:00.809995: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 10:15:00.838875: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-02-21 10:15:04.938073: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


0.8073432061693852
-0.46518010291595197


In [68]:
df = pd.DataFrame([[1, 2, 3], [4, 5, 6]], columns=['X', 'y', 'Z'])
train, y = standardize_dataset(df, prot_attr='Z', target='y')

print(y.index)

MultiIndex([(0, 3),
            (1, 6)],
           names=[None, 'Z'])
